# Import Libraries

In [2]:
import pandas as pd

# Obtain Data

Read recommender system data from csv file into df

In [3]:
df = pd.read_csv('Data/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


# Scrub Data

Perform df.info to check df column datatypes

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
userId       int64
movieId      int64
rating       float64
timestamp    int64
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


Check for missing values by calculating df entries by column

In [6]:
for col in df.columns:
    print(f'{col} Datapoints:', len(df[col].values))

userId Datapoints: 25000095
movieId Datapoints: 25000095
rating Datapoints: 25000095
timestamp Datapoints: 25000095


Check for abnormal values in dataset

In [23]:
print('userId Values:')
print(df.groupby('userId').first().index.sort_values())
print()
print('movieId Values:')
print(df.groupby('movieId').first().index.sort_values())
print()
print('rating Value Counts:')
print(df['rating'].value_counts())

userId Values:
Int64Index([     1,      2,      3,      4,      5,      6,      7,      8,
                 9,     10,
            ...
            162532, 162533, 162534, 162535, 162536, 162537, 162538, 162539,
            162540, 162541],
           dtype='int64', name='userId', length=162541)

movieId Values:
Int64Index([     1,      2,      3,      4,      5,      6,      7,      8,
                 9,     10,
            ...
            209145, 209147, 209151, 209153, 209155, 209157, 209159, 209163,
            209169, 209171],
           dtype='int64', name='movieId', length=59047)

rating Value Counts:
4.0    6639798
3.0    4896928
5.0    3612474
3.5    3177318
4.5    2200539
2.0    1640868
2.5    1262797
1.0     776815
1.5     399490
0.5     393068
Name: rating, dtype: int64


# Explore Data